# Counter Strikle Webscraper

> The following code scrapes liquidpedia (https://liquipedia.net/counterstrike/Main_Page), in order to make counter strikle (https://blast.tv/counter-strikle) puzzle-solving easier.

# Libs

In [1]:
import os
import csv
import time
from time import sleep
from bs4 import BeautifulSoup
from datetime import datetime

import numpy as np
import pandas as pd
#pd.set_option("display.max_rows", None)
from tqdm.notebook import tqdm
import requests
from urllib.request import quote

In [2]:
headers = {"User-Agent": "Counter-Strikle-Bot","Accept-Encoding": "gzip"}
base_url = "https://liquipedia.net/counterstrike/api.php?"
REGIONS = {"Europe" : ["Europe", "CIS"], "Americas" : ["Americas"],
           "Asia-Pacific" : ["Oceania", "Eastern_%26_Southern_Asia"]}
CSV_FILE = "players2.csv"
COLUMNS = ["NAME", "REAL NAME", "REGION", "NATIONALITY", "TEAM", "AGE", "WEAPON", 
           "MAJOR APPEARANCES", "EARNINGS", "LAST UPDATED"]

def fetch_page(page):
    url = base_url +"action=parse&format=json&page=" + page
    response = requests.get(url, headers)
    page_html = response.json()['parse']['text']['*']
    soup = BeautifulSoup(page_html,features="lxml")
    time.sleep(45)
    return soup

In [3]:
def fetch_majors():
    soup = fetch_page("Majors")
    event_elements = soup.find_all("div", {"class": "divRow tournament-card-premier"})
    return [event.find("b").find("a")["href"] for event in event_elements]

#fetch_majors()

In [4]:
def fetch_ids(region):
    page = fetch_page("Portal:Players/" + region)
    elements = [item.find_all("a")[1] for item in page.find_all("td")]
    name_dict = [element["title"].replace(" ","_") for element in elements]
    return name_dict

#fetch_ids("Europe")

In [5]:
def total_players():
    total = 0
    for major_region, sub_regions in REGIONS.items():
        for region in sub_regions:
            total += len(fetch_ids(region))
    return total

#total_players()

In [6]:
def fetch_player(player_name, region, major_list, debug=False):
    # pull player profile
    soup = fetch_page(player_name)
    output_dict = dict()
    
    # parse player information
    info_list = soup.find_all("div", {"class": "infobox-cell-2"})
    info_dict = {info_list[i].text[:-1] : info_list[i+1].text for i in range(0, len(info_list),2)}
    if debug:
        print(info_dict)
    # add name to dict
    output_dict["NAME"] = player_name
    output_dict["REGION"] = region
    
    # replace name with romanised name if needed
    if "Romanized Name" in info_dict:
        output_dict["REAL NAME"] = info_dict["Romanized Name"]
    else:
        output_dict["REAL NAME"] = info_dict["Name"]
    
    # get nationality
    if "Nationality" in info_dict:
        output_dict["NATIONALITY"] = info_dict["Nationality"][1:]
    
    # get team
    if "Team"  in info_dict:
        output_dict["TEAM"] = info_dict["Team"]
    
    # Get age
    if "Born" in info_dict:
        try:
            output_dict["AGE"] = datetime.strptime(info_dict["Born"][:-9].replace(",", ""), "%B %d %Y").strftime("%d/%m/%Y")
        except:
            pass
    
    # Get role
    if "Role" in info_dict:
        if "AWPer" in info_dict["Role"]:
            output_dict["WEAPON"] = "AWP"
    elif "Roles" in info_dict:
        if "AWPer" in info_dict["Roles"]:
            output_dict["WEAPON"] = "AWP"
    else:
        output_dict["WEAPON"] = "AK47"
            
            
    # Get major appearances
    try:
        soup = fetch_page(player_name + "/Results")
        event_elements = soup.find_all("tr", {"class": "valvemajor-highlighted"})
        event_name_list = [event.find("td", {"style": "text-align:left"}).find("a")["href"] for event in event_elements]
        event_name_list = [event for event in event_name_list if event in major_list]
        output_dict["MAJOR APPEARANCES"] = len(event_name_list)
    except:
        output_dict["MAJOR APPEARANCES"] = 0
    
    if "Approx. Total Winnings" in info_dict:
        output_dict["EARNINGS"] = int(info_dict["Approx. Total Winnings"].replace(",", "").replace("$", ""))
    else:
        output_dict["EARNINGS"] = 0
    
    output_dict["LAST UPDATED"] = datetime.now().strftime("%d/%m/%Y")
    return output_dict
    
#fetch_player("MingSir", "Eastern_%26_Southern_Asia", fetch_majors(), debug=True)
#fetch_player("AntO_oNNN", "CIS", fetch_majors(), debug=True)

In [7]:
def build_df():
    # Get dataframe to werite to
    if os.path.exists(CSV_FILE):
        playerdata = pd.read_csv(CSV_FILE)
    else:
        playerdata = pd.DataFrame(columns=COLUMNS)
        
    # Get list of all mayors
    major_list = fetch_majors()

    try:
        for major_region, sub_regions in tqdm(REGIONS.items()):
            for sub_region in tqdm(sub_regions, leave=False):
                region_ids = fetch_ids(sub_region)[:1]
                for player in tqdm(region_ids, leave=False):
                    if not (playerdata["NAME"].eq(player)).any():
                        row_data = fetch_player(player, major_region, major_list)
                        playerdata = pd.concat([playerdata, pd.DataFrame.from_records([row_data])], ignore_index=True)
    except Exception as e:
        print("Error:", e)
    
    playerdata.to_csv(CSV_FILE, index=False)
    return

build_df()

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
playerdata = pd.read_csv(CSV_FILE)

In [9]:
playerdata

,NAME,REAL NAME,REGION,NATIONALITY,TEAM,AGE,WEAPON,MAJOR APPEARANCES,EARNINGS,LAST UPDATED
0,Stikle-,Klesti Kola,Europe,Albania,Bad News Eagles,05/07/1998,NaN,0,1207,15/10/2022
1,AERO,Pavel Surmach,Europe,Russia,NaN,12/11/1988,NaN,0,3203,15/10/2022
2,A2z,Armeen Toussi,Americas,Canada,NaN,26/11/1990,NaN,0,13144,15/10/2022
3,ADK,Akram Smida,Asia-Pacific,Australia,NaN,08/03/2001,AWP,0,5836,15/10/2022
4,0i,Zhiwei Liu,Asia-Pacific,China,Invictus Gaming,18/06/1996,NaN,0,41606,15/10/2022
